In [3]:
repl.load(os.read(os.pwd/'git/'src/'main/'scala/"import.sc"))
repl.load(os.read(os.pwd/'git/'src/'main/'scala/"a.scala"))

Compiling /home/jovyan/data/(console)
Compiling /home/jovyan/data/(console)


In [4]:
import Diss._
import UaMeta._

import Diss._
import UaMeta._

In [5]:
import FileUtils._

import FileUtils._

In [6]:
  def hash(str: String, seed: Int): Int = {
    var res = 1
    for (c <- str) {
      res = res * seed + c.toInt
    }

    res
  }

  def extractUaFromPath(path: String) = {
    path.split("/").dropRight(1).last
  }

  lazy val autoRefSet = Set("ref", "реферат", "avto")

  def isAutoRefUA(path: String): Boolean = {
    val fileName = path.split("/").last.toLowerCase
    autoRefSet.exists(x => fileName.contains(x))
  }

  def convertNgUa(str: String): String = {
    str.filter(ch => ch == ' ' || (ch >= 'а' && ch <= 'Я') || (ch >= '\u0410' && ch <= '\u044F')).toUpperCase
  }

  //ID -> FileName
  lazy val badFiles: Set[(String, String)] =
    scala.io.Source.fromFile("all.ban.1", "UTF-8").getLines.map { line =>
      val split = line.split("\t")
      split.head -> split(1)
    }.toSet
//    scala.io.Source.fromFile("bad.files", "UTF-8").getLines.map { line =>
//      val split = line.split("\t")
//      split.head -> split.last
//    }.toSet
  Set.empty


defined function hash
defined function extractUaFromPath
autoRefSet: Set[String] = Set("ref", "реферат", "avto")
defined function isAutoRefUA
defined function convertNgUa
badFiles: Set[(String, String)] = Set(
  ("0514U000833", "dis.doc.txt"),
  ("0418U003307", "dis Заднєпровська.doc.txt"),
  ("0420U100505", "Дисертація_Ібрагим.docx.txt"),
  (
    "0410U000823",
    "дисер+автореф--текст дис 12.03--Дис[1]. Черкашина 11.03.10[1].doc.txt"
  ),
  ("0518U000044", "Dysser-Shpotyuk.pdf.txt"),
  ("0515U000605", "dis Курта С.А..doc.txt"),
  ("0821U100735", "Zlenko_diss.pdf.txt"),
  ("0508U000331", "Розділ 3.doc.txt"),
  ("0410U003049", "disser--dis3.doc.txt"),
  ("0417U001002", "dis .doc.txt"),
  ("0419U002578", "dis.doc.txt"),
  ("0407U001929", "ELENA.doc.txt"),
  ("0507U000152", "dis 3.doc.txt"),
  ("0821U100099", "dis.doc.txt"),
  ("0512U000112", "dis.pdf.txt"),
  ("0420U100433", "Дисертація_Савченко О.В..pdf.txt"),
  ("0512U000622", "Диссертация Шаповал ВМ--Дис 02052012.doc.txt"),
  ("0420U100302", "dis.docx.txt"),
  ("0418U001757", "Diss_Grytsenko.pdf.txt"),
  ("0415U001624", "dis.doc.txt"),
  ("0521U101520", "dis.doc.txt"),
  ("0498U000413", "dysertacia.doc.txt"),
  ("0821U100982", "dysertatsiia_2021_Pylypenko_Rozrobka_skladu.pdf.txt"),
  ("0404U001102", "dys01.doc.txt"),
  ("0418U000072", "дисертація_zhuravska_19.11.18_друк.doc.txt"),
  ("0420U100045", "дисертація Москальов 28.05.2020.pdf.txt"),
  ("0408U003646", "Оксана-дисер-посл.вар..doc.txt"),
  (
    "0822U100316",
    "1_Овчаренко_Метафора і метафоризація_філософсько-антропологічний вимір_14.12.21_остаточн.варіант.pdf.txt"
  ),
  ("0823U100716", "Dis_Ходирєва.pdf.txt"),
  ("0410U006583", "dis.doc.txt"),
  ("0521U100239", "Диссертация.pdf.txt"),
...
res6_6: Set[Nothing] = Set()

In [20]:
  def uaHashDir(id: Int): Unit = {
    val byId: Map[String, Vector[(String, FileName)]] = FileUtils.traverse(s"txt/$id/")
      .filter(_.endsWith(".txt")).map(p => extractUaFromPath(p) -> p).groupBy(_._1).mapValues(v => v.filterNot { case (_, path) => isAutoRefUA(path) })

    val iter: Iterator[String] = byId.toVector.iterator.flatMap { case (id, vec) =>

      val pages: Vector[String] = vec.filterNot{case (idd, path) =>
        badFiles.contains(idd -> path.split("/").last)}.flatMap { case (_, p) =>
        read(p).split(12.toChar)
      }

      val mainText: Vector[String] = pages.map(page => convertNgUa(page).split(" ").filter(_.nonEmpty).mkString(" ")).flatMap { page => page.split(" ").sliding(6).map(_.mkString(" ")) }.filter(_.size >= 20)
      
      mainText.flatMap { ng =>
        val h5 = hash(ng, 5)
        val h7 = hash(ng, 7)
        val sum = (h5.toLong << 32) + h7
        if ((scala.math.abs(sum) % 10) == 0) {
          Some(s"${h5}\t${h7}\t${id}")
        } else {
          None
        }
      }
    }

    FileUtils.write(s"ua$id.ua", iter)
  }


defined function uaHashDir

In [ ]:
uaHashDir(0)

In [ ]:
(1 to 999).par.foreach(uaHashDir)

In [ ]:
  val uaHashMap = scala.collection.mutable.Map[(Int, Int), Int]().withDefaultValue(0)


In [ ]:
  GroupIterator[String, String, Vector[String]](scala.io.Source.fromFile("ua.hash.sort").getLines(), { x => {
    val split = x.split("\t")
    split.take(2).mkString(" ") -> Vector(split.last)
  }
  }, _ ++ _).filter(x => x._2.size > 1 && x._2.size <= 7).foreach {case (_, vec) =>
    for { i <- vec.indices
      j <- i + 1 until vec.size
    } {
      val fst = vec(i)
      val snd = vec(j)

      val meta1 = getUaMetaByStringId(fst)
      val meta2 = getUaMetaByStringId(snd)

      val (i1, i2) = (meta1.year,  meta2.year) match {
        case (y1, y2) if y1 < y2 => idUaRevMap(meta1.id) -> idUaRevMap(meta2.id)
        case (y1, y2) if y1 > y2 => idUaRevMap(meta2.id) -> idUaRevMap(meta1.id)
        case (_, _) if meta1.id < meta2.id => idUaRevMap(meta1.id) -> idUaRevMap(meta2.id)
        case _ => idUaRevMap(meta2.id) -> idUaRevMap(meta1.id)
      }

      uaHashMap(i1 -> i2) = uaHashMap(i1 -> i2) + 1
    }
  }


In [ ]:
  // val iter = uaHashMap.filter(_._2 >= 7).iterator.map(x => s"${x._1._1}\t${x._1._2}\t${x._2}")
  // FileUtils.write("ua.hash.7.graph.1", iter)


In [ ]:

lazy val uaSuspect: Vector[((Int, Int), Int)] = uaHashMap.filter(x => x._2 >= 200 && x._2 <= 2000).toVector
  lazy val uaSuspectGrouped: Vector[(Int, Vector[((Int, Int), Int)])] = uaSuspect.groupBy(_._1._2).toVector.sortBy(_._2.map(_._2).sum)(Ordering[Int].reverse)


In [ ]:
uaSuspectGrouped.take(10).foreach(println)

In [ ]:
  getUaMetaById(130947)
  getUaMetaById(130951)


In [ ]:
  // val iter1 = uaSuspectGrouped.take(5000).map { case (id, vec: Vector[((Int, Int), Int)]) =>
val iter1 = uaSuspectGrouped.map { case (id, vec: Vector[((Int, Int), Int)]) =>
    val m = getUaMetaById(id)

    val sources = vec.sortBy(_._2)(Ordering[Int].reverse).take(4).map { case ((other, _), pop) =>
      val mm = getUaMetaById(other)
      List(mm.id, mm.fullDate, mm.author, mm.title, s"weight=${pop}").mkString("[", ", ", "]")
    }.mkString("[[*", ", ", "*]]")

    List(m.id.toString, m.fullDate, m.title, m.author, sources).mkString("\t")
  }.iterator

  FileUtils.write("ua.top.p200-2000.csv", iter1)


In [ ]:
        // lazy val uaSuspect1: Vector[((Int, Int), Int)] = uaHashMap.filter(x => x._2 >= 200).toVector
        // lazy val uaSuspectGrouped1Map: Map[Int, Vector[((Int, Int), Int)]] = uaSuspect1.groupBy(_._1._2)


In [ ]:
    val iter = scala.io.Source.fromFile("ua.hash.7.all").getLines.flatMap { line =>
      val split = line.split("\t")
      val fst = split(0).toInt
      val snd = split(1).toInt
      val size = split(2).toInt

      val fstMeta: UaMeta = getUaMetaById(fst)
      val sndMeta: UaMeta = getUaMetaById(snd)

      val (fa, fb) = (fstMeta.year, sndMeta.year) match {
        case (f, s) if f == s => "--" -> "--"
        case (f, s) if f < s => "->" -> "<-"
        case (f, s) => "<-" -> "->"
      }

      Vector(
        s"${fstMeta.id}\t${fa}\t${sndMeta.id}\t$size",
        s"${sndMeta.id}\t${fb}\t${fstMeta.id}\t$size"
      )
    }

    FileUtils.write("ua.hash.7.all.graph", iter)


In [ ]:
val allSpec = uaMetaVec.groupBy(_.spec).mapValues(_.size).toVector.sortBy(_._1)

In [ ]:
uaMetaVec.filter(_.spec == "").take(5)

In [ ]:
    lazy val naPravahText = Set("НА ПРАВАХ", "НА ПРАВАХ")
    lazy val disserText = Set("ДИСЕРТ", "ДИСCЕРТ", "ДИССЕРТАЦИЯ")
    lazy val monoText = Set("МОНОГРАФІЯ")
    lazy val yearText = (2000 to 2024).toList.map(_.toString)

    def isTitle(page: String): Boolean = {
      disserText.exists(x => page.contains(x)) || naPravahText.exists(x => page.contains(x))
    }


In [ ]:
    def uaBanCheck(id: Int): Unit = {
      val byId: Map[String, Vector[(String, FileName)]] = FileUtils.traverse(s"txt/$id/")
        .filter(_.endsWith(".txt")).map(p => extractUaFromPath(p) -> p).groupBy(_._1).mapValues(v => v.filter { case (_, path) => !isAutoRefUA(path) })

      def badTitle(meta: UaMeta, fstPage: String, years: List[String]): Boolean = {
        !(fstPage.filterNot(_ == ' ').contains(meta.spec) && years.exists(y => fstPage.contains(y)))
      }

      def isMono(fstPage: String): Boolean = {
        monoText.exists(x => fstPage.contains(x)) && yearText.exists(y => fstPage.contains(y))
      }

      val iter: Iterator[String] = byId.toVector.iterator.flatMap { case (id, vec) =>
        val meta = getUaMetaByStringId(id)

        vec.filterNot(x => isAutoRefUA(x._2)).flatMap { case (_, p) =>
          val fileSize = FileUtils.fileSize(p)
          if (fileSize > 400000) {
            val fstPage = read(p).split(12.toChar).head.toUpperCase

            if (isTitle(fstPage)) {
              val years = List(meta.year, meta.year + 1, meta.year - 1).map(_.toString)
              if (badTitle(meta, fstPage, years)) {
                Some(s"$id\t${p.split("/").last}\t${meta.spec}\t${meta.year}\t${fileSize}\tDiss")
              } else {
                None
              }
            } else {
              if (isMono(fstPage)) {
                Some(s"$id\t${p.split("/").last}\t${meta.spec}\t${meta.year}\t${fileSize}\tMono")
              } else {
                Some(s"$id\t${p.split("/").last}\t${meta.spec}\t${meta.year}\t${fileSize}\tUnknown")
              }
            }
          } else {
            None
          }
        }

      }

      FileUtils.write(s"ua$id.ban", iter)
    }


In [ ]:
uaBanCheck(0)

In [ ]:
(1 to 999).par.foreach(uaBanCheck)

In [22]:
  def ua2RuHashDir(id: Int): Unit = {
    val byId: Map[String, Vector[(String, FileName)]] = FileUtils.traverse(s"tr_ru/$id/")
      .filter(_.endsWith(".txt")).map(p => extractUaFromPath(p) -> p).groupBy(_._1).mapValues(v => v.filterNot { case (_, path) => isAutoRefUA(path) })

    val iter: Iterator[String] = byId.toVector.iterator.flatMap { case (id, vec) =>

      val pages: Vector[String] = vec.filterNot{case (idd, path) =>
        badFiles.contains(idd -> path.split("/").last)}.flatMap { case (_, p) =>
        read(p).split(12.toChar)
      }

      val mainText: Vector[String] = pages.map(page => convertNgUa(page).split(" ").filter(_.nonEmpty).mkString(" ")).flatMap { page => page.split(" ").sliding(6).map(_.mkString(" ")) }.filter(_.size >= 20)
      
      mainText.flatMap { ng =>
        val h5 = hash(ng, 5)
        val h7 = hash(ng, 7)
        val sum = (h5.toLong << 32) + h7
        if ((scala.math.abs(sum) % 10) == 0) {
          Some(s"${h5}\t${h7}\tR${id}")
        } else {
          None
        }
      }
    }

    FileUtils.write(s"rua$id.rua", iter)
  }


defined function ua2RuHashDir

In [23]:
ua2RuHashDir(0)

In [24]:
(0 to 999).par.foreach(ua2RuHashDir)

In [13]:
  def ruTextHash(fileName: String, id: String): Iterator[String] = {
    val bibList = List[String](
      "СПИСОК ЛИТЕРАТУРЫ", "СПИСОК ИСПОЛЬЗОВАННОЙ", "СПИСОК ИСПОЛЬЗУЕМОЙ", "СПИСОКЛИТЕРАТУРЫ", "УКАЗАТЕЛЬ ЛИТЕРАТУРЫ",
      "СПИСОК ЦИТИРУЕМОЙ", "ИСПОЛЬЗОВАННАЯ ЛИТЕРАТУРА", "БИБЛИОГРАФИЧЕСКИЙ СПИСОК", "СПИСОК ИСПОЛЬЗОВАННЫХ",
      "БИБЛИОГРАФИЧЕСКИЙ", "БИБЛИОГРАФИЯ", "СПИСОК ИСТОЧНИКОВ", "ЛИТЕРАТУРА")

    val endList = List[String]("ВЫВОДЫ", "ЗАКЛЮЧЕНИЕ")

    import scala.util.Try
    Try {
      val a: Array[String] = {
        import java.io._
        val br = new BufferedReader(new InputStreamReader(
          new FileInputStream(fileName), "UTF-8"))
        val res = br.lines().toArray.toList.map(_.toString).mkString(" ").split(12.toChar)
        br.close()
        res
      }

      val zip: Array[(String, Int)] = a.zipWithIndex
      val rev = zip.reverse

      val bib: Option[Int] = rev.take((rev.size * 3 / 4).toInt).find { case (str, _) =>
        val list = str.take(100).toUpperCase.filter(x => x.isLetter || x == ' ').split(" ").filter(_.nonEmpty)
        val head = list.mkString(" ")
        bibList.exists(prefix => head.startsWith(prefix))
      }.map(_._2)

      val end: Option[Int] = rev.take((rev.size * 3 / 4).toInt).find { case (str, _) =>
        val list = str.take(100).toUpperCase.filter(x => x.isLetter || x == ' ').split(" ").filter(_.nonEmpty)
        val head = list.mkString(" ")
        endList.exists(prefix => head.startsWith(prefix))
      }.map(_._2)

      val endPos = (bib, end) match {
        case (Some(b), Some(e)) => scala.math.min(b, e)
        case (None, Some(e)) => e
        case (Some(b), None) => b
        case _ => a.size - 10
      }

      val pages: List[String] = zip.toList.slice(5, endPos).map(_._1)

      val mainText: Vector[String] = pages.map(page => convertNgUa(page).split(" ").filter(_.nonEmpty).mkString(" ")).flatMap { page => page.split(" ").sliding(6).map(_.mkString(" ")) }.filter(_.size >= 20).toVector

      mainText.flatMap { ng =>
        val h5 = hash(ng, 5)
        val h7 = hash(ng, 7)
        val sum = (h5.toLong << 32) + h7
        if ((scala.math.abs(sum) % 10) == 0) {
          Some(s"${h5}\t${h7}\t${id}")
        } else {
          None
        }
      }.iterator
    }.getOrElse(Iterator.empty)
  }

  def ruDirHashHandle(id: Int): Unit = {
    val iter = FileUtils.traverse(s"data/d/$id").iterator.flatMap { fileName =>
      val i = fileName.split("/").last
      val rgbId: Int = all(i)
      ruTextHash(fileName, s"D$rgbId")
    }

    FileUtils.write(s"ru$id.hash", iter)
  }


cell14.sc:47: not found: value hash
        val h5 = hash(ng, 5)
                 ^
cell14.sc:48: not found: value hash
        val h7 = hash(ng, 7)
                 ^
Compilation Failed

In [ ]:
ruDirHashHandle(0)

In [ ]:
(1 to 999).par.foreach(ruDirHashHandle)

In [ ]:
  val map = scala.collection.mutable.Map[(String, String), Int]().withDefaultValue(0)

  GroupIterator[String, String, Vector[String]](scala.io.Source.fromFile("ua.ru.hash.sort.hh").getLines, { line =>
    val split = line.split("\t")
    split.take(2).mkString("\t") -> Vector(split.last)
  }, _ ++ _).filter(x => x._2.size >= 2 && x._2.size <= 10).map(_._2).filter { vec =>
    vec.count(_.startsWith("D")) >= 1 && vec.count(s => s.head.isDigit) >= 1
  }.foreach { vec =>
    val ru = vec.filter(_.startsWith("D"))
    val ua = vec.filter(!_.startsWith("D"))

    for { r <- ru
      u <- ua
      p = r -> u
    } {
      map(p) = map(p) + 1
    }
  }

  val iter = map.filter(_._2 >= 7).iterator.map { x =>
    s"${x._1._1}\t${x._1._2}\t${x._2}"
  }

  FileUtils.write("ru.ua.text.hash.graph", iter)


In [ ]:
    val iter = scala.io.Source.fromFile("ru.ua.text.hash.graph").getLines.flatMap { line =>
      val split = line.split("\t")
      val size = split.last.toInt

      if (size >= 100) {
        val ru = getDissMeta(split.head.drop(1).toInt)
        val ua = getUaMetaByStringId(split(1))

        Some(ru, ua, size)
      } else {
        None
      }
    }.toVector.sortBy(x => x._1.y)(Ordering[Int].reverse).toIterator.flatMap { case (ru, ua, size) =>
      if (ru.y >= ua.year) {
        val list = List[String](ru.id, ru.year, ru.author, ru.toString, ua.id, ua.year.toString, ua.author, ua.toString, size.toString)
        Some(s"${list.mkString("\t")}")
      } else {
        None
      }
    }

    FileUtils.write("ua2ru.text.csv", iter)


In [ ]:
    val iter = scala.io.Source.fromFile("ru.ua.text.hash.graph").getLines.flatMap { line =>
      val split = line.split("\t")
      val size = split.last.toInt

      if (size >= 100) {
        val ru = getDissMeta(split.head.drop(1).toInt)
        val ua = getUaMetaByStringId(split(1))

        Some(ru, ua, size)
      } else {
        None
      }
    }.toVector.sortBy(x => x._2.year)(Ordering[Int].reverse).toIterator.flatMap { case (ru, ua, size) =>
      if (ru.y <= ua.year) {
        val list = List[String](ua.id, ua.year.toString, ua.author, ua.toString, ru.id, ru.year, ru.author, ru.toString, size.toString)
        Some(s"${list.mkString("\t")}")
      } else {
        None
      }
    }

    FileUtils.write("ru2ua.text.csv", iter)


In [25]:
  val map = scala.collection.mutable.Map[(String, String), Int]().withDefaultValue(0)

  GroupIterator[String, String, Vector[String]](scala.io.Source.fromFile("rua.ru.hash.sort.hh").getLines, { line =>
    val split = line.split("\t")
    split.take(2).mkString("\t") -> Vector(split.last)
  }, _ ++ _).filter(x => x._2.size >= 2 && x._2.size <= 10).map(_._2).filter { vec =>
    vec.count(_.startsWith("D")) >= 1 && vec.count(_.startsWith("R")) >= 1
  }.foreach { vec =>
    val ruTr = vec.filter(_.startsWith("R"))
    val ru = vec.filter(_.startsWith("D"))

    for {r <- ruTr
         u <- ru
         p = r -> u
         } {
      map(p) = map(p) + 1
    }
  }

  val iter = map.filter(_._2 >= 7).iterator.map { x =>
    s"${x._1._1}\t${x._1._2}\t${x._2}"
  }

  FileUtils.write("rua.ru.hash.hh.graph", iter)


map: mutable.Map[(String, String), Int] = Map(
  ("R0410U005223", "D147628") -> 1,
  ("R0418U002271", "D898557") -> 1,
  ("R0415U003860", "D637317") -> 1,
  ("R0415U000311", "D959275") -> 1,
  ("R0419U004814", "D455917") -> 1,
  ("R0515U000560", "D806309") -> 1,
  ("R0410U003726", "D599283") -> 1,
  ("R0504U000421", "D71972") -> 1,
  ("R0417U003801", "D810692") -> 1,
  ("R0519U000255", "D605242") -> 1,
  ("R0410U001177", "D634223") -> 1,
  ("R0416U003063", "D236213") -> 1,
  ("R0521U100081", "D645348") -> 2,
  ("R0420U102221", "D485141") -> 1,
  ("R0821U101785", "D891501") -> 1,
  ("R0406U000876", "D253359") -> 2,
  ("R0409U001433", "D785095") -> 1,
  ("R0411U003858", "D976364") -> 1,
  ("R0415U001352", "D780757") -> 1,
  ("R0505U000023", "D363266") -> 1,
  ("R0406U002168", "D44303") -> 1,
  ("R0410U004943", "D53395") -> 1,
  ("R0403U002358", "D95318") -> 1,
  ("R0405U003324", "D141405") -> 1,
  ("R0518U000720", "D75517") -> 1,
  ("R0405U001721", "D652745") -> 1,
  ("R0521U100820", "D7

In [26]:
    val ordering2 = new Ordering[(MetaData2, UaMeta, Int)] {
      override def compare(x: (MetaData2, UaMeta, Int), y: (MetaData2, UaMeta, Int)): Int = {
        val c1 = -(x._1.year compareTo y._1.year)
        val c2 = x._1.id compareTo y._1.id
        (c1, c2) match {
          case (a, _) if a != 0 => a
          case (_, a) => a
        }
      }
    }

    val iter = scala.io.Source.fromFile("rua.ru.hash.hh.graph").getLines.flatMap { line =>
      val split = line.split("\t")
      val size = split.last.toInt

      if (size >= 7) {
        val ru = getDissMeta(split(1).drop(1).toInt)
        val ua = getUaMetaByStringId(split.head.drop(1))

        Some(ru, ua, size)
      } else {
        None
      }
    }.toVector.sorted(ordering2).toIterator.flatMap { case (ru, ua, size) =>
      if (ru.y >= ua.year) {
        val list = List[String](ru.id, ru.year, ru.author, ru.toString, ua.id, ua.year.toString, ua.author, ua.toString, size.toString)
        Some(s"${list.mkString("\t")}")
      } else {
        None
      }
    }

    FileUtils.write("trans.ua2ru.from7.4of4.text.csv", iter)


ordering2: Object with Ordering[(MetaData2, UaMeta, Int)] = ammonite.$sess.cell26$Helper$$anon$1@608b381c
iter: Iterator[String] = empty iterator

In [27]:
    val ordering1 = new Ordering[(MetaData2, UaMeta, Int)] {
      override def compare(x: (MetaData2, UaMeta, Int), y: (MetaData2, UaMeta, Int)): Int = {
        val c1 = -(x._2.year compareTo y._2.year)
        val c2 = x._2.id compareTo y._2.id
        (c1, c2) match {
          case (a, _) if a != 0 => a
          case (_, a) => a
        }
      }
    }

    val iter = scala.io.Source.fromFile("rua.ru.hash.hh.graph").getLines.flatMap { line =>
      val split = line.split("\t")
      val size = split.last.toInt

      if (size >= 7) {
        val ru = getDissMeta(split(1).drop(1).toInt)
        val ua = getUaMetaByStringId(split.head.drop(1))

        Some(ru, ua, size)
      } else {
        None
      }
    }.toVector.sorted(ordering1).toIterator.flatMap { case (ru, ua, size) =>
      if (ru.y <= ua.year) {
        val list = List[String](ua.id, ua.year.toString, ua.author, ua.toString, ru.id, ru.year, ru.author, ru.toString, size.toString)
        Some(s"${list.mkString("\t")}")
      } else {
        None
      }
    }

    FileUtils.write("trans.ru2ua.from7.4of4.text.csv", iter)


ordering1: Object with Ordering[(MetaData2, UaMeta, Int)] = ammonite.$sess.cell27$Helper$$anon$1@223b787a
iter: Iterator[String] = empty iterator

In [ ]:
  //   val lookingFor = Vector("Національний авіаційний університет".toUpperCase(), "national aviation university".toUpperCase(), 
  // "национальный авиационный университет".toUpperCase())


  // def uaFind(id: Int): Unit = {
  //   val byId: Map[String, Vector[(String, FileName)]] = FileUtils.traverse(s"txt/$id/")
  //     .filter(_.endsWith(".txt")).map(p => extractUaFromPath(p) -> p).groupBy(_._1).mapValues(v => v.filter { case (_, path) => isAutoRefUA(path) })

  //   val iter: Iterator[String] = byId.toVector.iterator.flatMap { case (id, vec) =>
  //     vec.iterator.filterNot { case (idd, path) =>
  //       badFiles.contains(idd -> path.split("/").last)
  //     }.flatMap { case (_, p) =>
  //       val page = read(p).split(12.toChar).headOption.getOrElse("").replace('\n', ' ').split(" ").filter(_.nonEmpty).mkString(" ").toUpperCase()
  //       if (lookingFor.exists(look => page.contains(look))) {
  //         Some(extractUaFromPath(p))
  //       } else {
  //         None
  //       }
  //     }
  //   }

  //   FileUtils.write(s"look$id.search", iter)
  // }


In [40]:
uaFind(0)

In [41]:
(0 to 999).par.foreach(uaFind)

In [49]:
  val autoref = "Автореферат".toUpperCase
  val disser = Set("Дисертація".toUpperCase, "диссертация".toUpperCase)

  def uaAutoMetaGen(id: Int): Unit = {
    val byId: Map[String, Vector[(String, FileName)]] = FileUtils.traverse(s"txt/$id/")
      .filter(_.endsWith(".txt")).map(p => extractUaFromPath(p) -> p).groupBy(_._1).mapValues(v => v.filter { case (_, path) => isAutoRefUA(path) })

    val dissById: Map[String, Vector[(String, FileName)]] = FileUtils.traverse(s"txt/$id/")
      .filter(_.endsWith(".txt")).map(p => extractUaFromPath(p) -> p).groupBy(_._1).mapValues(v => v.filter { case (_, path) => !isAutoRefUA(path) })

    val iter: Iterator[String] = byId.toVector.iterator.flatMap { case (id, vec) =>
      vec.iterator.filterNot { case (idd, path) =>
        badFiles.contains(idd -> path.split("/").last)
      }.flatMap { case (_, p) =>
        val pages: Array[String] = read(p).split(12.toChar).take(2)
        if (pages.map(_.toUpperCase()).exists(p => {
          p.contains(autoref)
        })) {
          val id = extractUaFromPath(p)
          val meta = getUaMetaByStringId(id)

          Some(s"$id\tdiss\t${meta.spec}\t${meta.year}\t${pages.map(pageNorm).mkString("\t")}")
        } else {
          None
        }
      }
    }

    val dissiter: Iterator[String] = dissById.toVector.iterator.flatMap { case (id, vec) =>
      vec.iterator.filterNot { case (idd, path) =>
        badFiles.contains(idd -> path.split("/").last)
      }.flatMap { case (_, p) =>
        val page = read(p).split(12.toChar).take(1).headOption.getOrElse("")
        val pp = page.toUpperCase
        if (disser.exists(dis => pp.contains(dis))) {
          val id = extractUaFromPath(p)
          val meta = getUaMetaByStringId(id)

          Some(s"$id\tautoref\t${meta.spec}\t${meta.year}\t${page}")
        } else {
          None
        }
      }
    }

    FileUtils.write(s"aumeta$id.aumeta", iter ++ dissiter)
  }


autoref: String = "АВТОРЕФЕРАТ"
disser: Set[String] = Set("ДИСЕРТАЦІЯ", "ДИССЕРТАЦИЯ")
defined function uaAutoMetaGen

In [50]:
uaAutoMetaGen(0)

In [51]:
(0 to 999).par.foreach(uaAutoMetaGen)